In [482]:

import pandas as pd

import plotly
import plotly.graph_objects as go
import plotly.express as px



for sheet_name in ['KEM-PKE','DS']:
    data = pd.read_excel('/Users/kamyar/Downloads/PQC_Round_3_Parameter_Sets.xlsx', sheet_name=sheet_name).sort_values(by=['Candidate', 'Level'])


    subfamilies = data['Subfamily'].unique()

    x_column = 'KEM Public Key Size' if sheet_name == 'KEM-PKE' else 'Public Key Size'
    y_column = 'KEM Ciphertext Size' if sheet_name == 'KEM-PKE' else 'Signature Size'

    subfamily_color_map = {k:v for k,v in zip(subfamilies.tolist(), px.colors.qualitative.Plotly) }

    level_to_symbols_map = {1:1, 2:2, 3:0, 4:3, 5:4}

    level_to_size_map = {i:i*1 + 10 for i in range(1,6)}

    fig = go.Figure()
    # fig = px.scatter(data, 
    #     x=x_column, y=y_column,
    #     log_x=True, log_y=True, 
    #     color="Candidate",
    #     opacity=0.8,
    #     size_max=18,
    #     #facet_col='',
    #     symbol="Level",
    #     size="Level",
    #     width=1200, height=800, 
    #     color_discrete_sequence=px.colors.qualitative.T10, 
    #     hover_name='Candidate',
    #     # title='KEM Ciphertext size vs Public Key size',
    #     labels={
    #         x_column: "Public Key (bytes)",
    #         y_column: "Ciphertext (bytes)",
    #         "Level": "Security Level"
    #     },

    # )

    candidates = data['Candidate'].unique()
    levels = range(1,6)

    candidate_colors = data['Candidate'].map( {v:k for k,v in enumerate(candidates)} )

    def cnd_traces(showlegend):
        for cnd_idx,cnd in enumerate(candidates):
            trace_data = data[(data['Candidate'] == cnd)]
            if not 1 in trace_data['Level'].values:
                d1 = trace_data.iloc[0]
                d1['Level'] = 1
                trace_data.loc[-1] = d1  # adding a row
                trace_data.index = trace_data.index + 1  # shifting index
                trace_data = trace_data.sort_index()  # sorting by index
            fig.add_trace(
                go.Scatter(
                    x=trace_data[x_column], y=trace_data[y_column],
                    mode = "markers",
                    showlegend=showlegend,
                    legendgroup = cnd,
                    hovertext = None,
                    name=f"{cnd} ({trace_data.iloc[0]['Subfamily']})",
                    text = trace_data['Parameter Set'],
                    hoverinfo = 'x+y+text',
                    hovertemplate ='<b>%{text}</b> <br>' + '<br> Public Key: %{x:,f} <br> ' + ('Ciphertext' if sheet_name == 'KEM-PKE' else 'Signature') + ': %{y:,f}<extra></extra>',
                    # color_discrete_sequence=px.colors.qualitative.T10, 
                    # hover_name='Candidate',
                    marker_size = 12, #[ s*2 + 12  for s in  data['Level']],
                    marker_symbol = trace_data['Level'].map(level_to_symbols_map),
                    marker_color= px.colors.qualitative.T10[cnd_idx],
                    # marker_size = trace_data['Level'].map(level_to_size_map),
                    marker=dict(
                        # size=trace_data['Level'],
                        # sizemin=7,
                        # color=cnd_idx,
                        # sizeref=0.5,
                        opacity=0.9,
                        # line=dict(width=0.5, color='black')
                    ),
                )
            )

    cnd_traces(True)



    for lvl in levels:
        trace_data = data[ (data['Level'] == lvl)]
        fig.add_trace(
            go.Scatter(
                x=trace_data[x_column], y=trace_data[y_column],
                mode = "markers",
                showlegend=True,
                legendgroup=lvl,
                # opacity=0.1,
                name=f"Security Level {lvl}",
                # visible='legendonly',
                hoverinfo='none',
                hoverlabel = dict(namelength=0),
                marker_color='Black',
                marker_size = 12, #[ s*2 + 12  for s in  data['Level']],
                marker_symbol = trace_data['Level'].map({k:v+100 for k,v in level_to_symbols_map.items() }),
                # text = trace_data['Parameter Set'],
                # marker_size = trace_data['Level'].map(level_to_size_map),
                marker=dict(
                    # size=trace_data['Level'],
                    # sizemin=7,
                    # color=cnd_idx,
                    # sizeref=0.5,
                    # opacity=0.95,
                    # line=dict(width=1, color='orange')
                ),
            )
        )
    
    cnd_traces(False)

    fig.update_traces(marker=dict(line=dict(width=0.5,
                                color='black')),
                                selector=dict(mode='markers'))


    fig.update_layout(
        xaxis_type="log", yaxis_type="log",
        width=1200, height=800,
        xaxis = dict(
            tickmode = 'array',
            tickvals =[ 2**(2+i) for i in range(0,24) ],
            tickformat = ',f'
        ),
        yaxis = dict(
            tickmode = 'array',
            tickvals =[ 2**(2+i) for i in range(0,28) ],
            tickformat = ',f'
        ),
    )


    fig.update_xaxes(
        tickangle=45,
        title_text = 'Public Key (bytes)',
        title_font=dict(
            size=22, 
            # family='Courier',
            # color='crimson'
        ),
        tickfont=dict(
            # family='Rockwell', 
            # color='crimson', 
            size=14
        ),

    )

    fig.update_yaxes(
        title_text = 'Ciphertext (bytes)' if sheet_name == 'KEM-PKE' else 'Signature Size (bytes)',
        title_font=dict(
            size=22,
            # family='Courier', 
            # color='crimson'
        ),
        tickfont=dict(
            # family='Rockwell', 
            # color='crimson', 
            size=14
        ),
    )

    # fig.update_layout(
    #     margin=dict(l=0, r=0, t=0, b=0),
    # #     paper_bgcolor="LightSteelBlue",
    # )

    config={
        'displaylogo': False,
        'displayModeBar': True,
        'modeBarButtonsToRemove': ['hoverClosest3d', 'hoverClosestCartesian', 'hoverClosestGeo' , 'hoverClosestGl2d', 'hoverClosestPie' , 
            'lasso2d','toggleSpikelines',' hoverClosestCartesian', 'hoverCompareCartesian'], 
        'responsive': True
    }

    fig.update_layout(
        legend=dict(
            yanchor="top",
            y=0.96,
            xanchor="right",
            x=0.97,
            # itemsizing='constant',
            # traceorder='grouped',
            borderwidth=2,
            bordercolor='#ffffff',
            itemsizing='constant',
            # title_font_family="Times New Roman",
            # title_font_size=24,
            # title = 'Legend',
            font=dict(
                # family="Courier",
                size=14,
                color="black"
            ),
        )

        # margin=dict(l=1,r=1,t=1,b=1),
    )

    plot_name = f'plot_{sheet_name.lower()}'

    fig.show(config=config)
    fig.write_html(f"{plot_name}.html", include_plotlyjs='directory',config=config)
    fig.write_image(f"{plot_name}.svg")
    fig.write_image(f"{plot_name}.pdf")

In [466]:
from plotly.subplots import make_subplots

for sheet_name in ['KEM-PKE','DS']:
    data = pd.read_excel('/Users/kamyar/Downloads/PQC_Round_3_Parameter_Sets.xlsx', sheet_name=sheet_name).sort_values(by=['Candidate', 'Level',])


    subfamilies = data['Subfamily'].unique()

    x_column = 'KEM Public Key Size' if sheet_name == 'KEM-PKE' else 'Public Key Size'
    y_column = 'KEM Ciphertext Size' if sheet_name == 'KEM-PKE' else 'Signature Size'



    fig = make_subplots(rows=1, cols=5)
    # fig = go.Figure()
    fig = px.scatter(data, 
        x=x_column, y=y_column,
        log_x=True, log_y=True, 
        color="Candidate",
        opacity=1,
        size_max=12,
        # size=[12 for _ in range(0, len(data))],
        facet_col="Level",
        # symbol = data['Level'].map(level_to_symbols_map),
        # width=1600, height=800,
        facet_col_spacing=0.006,
        color_discrete_sequence=px.colors.qualitative.T10, 
        hover_name='Candidate',
        # name=f"{cnd} ({trace_data.iloc[0]['Subfamily']})",
        # title='KEM Ciphertext size vs Public Key size',
        labels={
            x_column: "Public Key (bytes)" ,
            y_column: 'Ciphertext (bytes)' if sheet_name == 'KEM-PKE' else 'Signature Size (bytes)',
            "Level": "Security Level",
        },
        # category_orders={'Candidate': ['SIKE', 'CRYSTALS-KYBER', 'NTRU', 'NTRU Prime', 'BIKE', 'HQC', 'FrodoKEM', 'Classic McEliece'], 'Level': ['3','1','2','4','5']}
        # template='seaborn',

    )

    # candidates = data['Candidate'].unique()
    # levels = range(1,6)

    # candidate_colors = data['Candidate'].map( {v:k for k,v in enumerate(candidates)} )

    # for lvl in levels:
    #     for cnd_idx,cnd in enumerate(candidates):
    #         trace_data = data[(data['Level'] == lvl) & (data['Candidate'] == cnd)]
    #         if len(trace_data) > 0:
    #             fig.add_trace(
    #                 go.Scatter(
    #                     x=trace_data[x_column], y=trace_data[y_column],
    #                     mode = "markers",
    #                     showlegend=True,
    #                     legendgroup=cnd,
    #                     name=f"{cnd} ({trace_data.iloc[0]['Subfamily']})",
    #                     # color_discrete_sequence=px.colors.qualitative.T10, 
    #                     # hover_name='Candidate',
    #                     marker_size = 12, #[ s*2 + 12  for s in  data['Level']],
    #                     marker_symbol = trace_data['Level'].map(level_to_symbols_map),
    #                     marker_color= px.colors.qualitative.T10[cnd_idx],
    #                     text = trace_data['Parameter Set'],
    #                     # marker_size = trace_data['Level'].map(level_to_size_map),
    #                     marker=dict(
    #                         # size=trace_data['Level'],
    #                         # sizemin=7,
    #                         # color=cnd_idx,
    #                         # sizeref=0.5,
    #                         opacity=1,
    #                         line=dict(width=0.5, color='black')
    #                     ),
    #                 ),
    #                 row=1, col=lvl
    #             )


    xaxis = dict(
            tickmode = 'array',
            tickvals =[ 2**(i) for i in range(6,28,2) ],
            tickformat = ',f'
    )

    yaxis = dict(
            tickmode = 'array',
            tickvals =[ 2**(3+i) for i in range(0,18) ],
            tickformat = ',f'
        )

    fig.update_layout(
        # xaxis_type="log", yaxis_type="log",
        width=1600, height=800,
        xaxis = xaxis,
        xaxis2 = xaxis,
        xaxis3 = xaxis,
        xaxis4 = xaxis,
        xaxis5 = xaxis,
        yaxis = yaxis,
        yaxis2 = yaxis,
        yaxis3 = yaxis,
        yaxis4 = yaxis,
        yaxis5 = yaxis,

    )

    fig.update_traces(marker=dict(size=12,line=dict(width=0.5,
                                        color='black')),
                                        selector=dict(mode='markers'))

    fig.for_each_annotation(
        lambda a: a.update(
            text=' '.join(a.text.split("=")),
            font=dict(
                # family="Courier",
                size=20,
                color="black"
            ),
        )
    )

    fig.update_xaxes(
        tickangle=45,
        # title_text = 'Public Key (bytes)',
        title_standoff = 0,
        title_font=dict(
            size=20, 
            # family='Courier',
            # color='crimson'
        ),
        tickfont=dict(
            # family='Rockwell', 
            # color='crimson', 
            size=12
        ),

    )

    fig.update_yaxes(
        # title_text = 'Ciphertext (bytes)' if sheet_name == 'KEM-PKE' else 'Signature Size (bytes)',
        title_standoff = 0,
        title_font=dict(
            size=20,
            # family='Courier', 
            # color='crimson'
        ),
        tickfont=dict(
            # family='Rockwell', 
            # color='crimson', 
            size=12
        ),
    )

    # fig.update_layout(
    #     margin=dict(l=0, r=0, t=0, b=0),
    # #     paper_bgcolor="LightSteelBlue",
    # )

    config={
        'displaylogo': False,
        'displayModeBar': True,
        'modeBarButtonsToRemove': ['hoverClosest3d', 'hoverClosestCartesian', 'hoverClosestGeo' , 'hoverClosestGl2d', 'hoverClosestPie' , 
            'lasso2d','toggleSpikelines',' hoverClosestCartesian', 'hoverCompareCartesian'], 
        'responsive': True
    }

    fig.update_layout(
        legend=dict(
            yanchor="middle",
            y=0.60,
            xanchor="right",
            x=0.99,
            # itemsizing='constant',
            # traceorder='reversed+grouped',
            borderwidth=1,
            bordercolor='#ffffff',
            # title_font_family="Times New Roman",
            title_font_size=20,
            title = '',
            font=dict(
                # family="Courier",
                size=14,
                color="black"
            ),
        )
    )

    plot_name = f'plot_facets_{sheet_name.lower()}'

    fig.show(config=config)
    fig.write_html(f"{plot_name}.html", include_plotlyjs='directory',config=config)
    fig.write_image(f"{plot_name}.svg")
    fig.write_image(f"{plot_name}.pdf")

In [476]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import pandas as pd
init_notebook_mode(connected=True)

feed = pd.DataFrame({'AAPL': [100, 120, 130, 120, 150],
                     'MSFT': [80, 70, 90, 100, 99]}, index = pd.date_range('2018-01-01', '2018-01-05'))

trans = pd.DataFrame({'Symbol': ['AAPL', 'AAPL', 'MSFT', 'MSFT'],
                        'Amount': [2, 1, 3, 1],
                        'Action': ['buy', 'sell', 'buy', 'sell'],
                        'Price': [120, 150, 70, 100],
                        'Date': [pd.to_datetime(i) for i in ['2018-01-02', '2018-01-05', '2018-01-02', '2018-01-04']]
                       })

data = []

# Add "All" button
all_but = dict(label = 'All',
               method = 'update',
                  args = [{'visible': [True]},
                          {'title': 'all'}
                           ])

buttonlist = [all_but]
for i, col in enumerate(feed.columns.tolist()):
    stock = go.Scatter(x=feed.index,
                       y=feed.loc[:, col],
                       name=col,
                       legendgroup='stock'
                      )
    # actions
    success = trans.loc[trans.Symbol == col]
    
    # Buy
    success_buy = success[success.Action == 'buy']
    action_buy = go.Scatter(x=success_buy.Date,
                         y=success_buy.Price,
                         name='Buy',
                         mode = 'markers',
                         text = ['Amount ' + str(amount) for amount in success_buy.Amount],
                         hoverinfo = 'text',
                         showlegend=False,
                         marker = dict(
                             size=10,
                             color = 'green'
                         ),
                         legendgroup='Buy',
                        )
    
    # Sell
    success_sell = success[success.Action == 'sell']
    action_sell = go.Scatter(x=success_sell.Date,
                         y=success_sell.Price,
                         name='Sell',
                         mode = 'markers',
                         text = ['Amount ' + str(amount) for amount in success_sell.Amount],
                         hoverinfo = 'text',
                         showlegend=False,
                         marker = dict(
                             size=10,
                             color = 'red'
                         ),
                         legendgroup='Sell',
                        )
    data.append(stock)
    data.append(action_buy)
    data.append(action_sell)
    button = dict(label = col,
                  method = 'update',
                  args = [{'visible': [col == i for i in feed.columns.tolist() for _ in range(3)] + [True, True]},
                          {'title': 'title' + col}
                           ])
    buttonlist.append(button)

# Add single buy and sell traces for legend
data.append(go.Scatter(x=[None], y=[None], mode='markers',
                       marker=dict(size=10, color='green'),
                       legendgroup='Buy', showlegend=True, name='Buy'))

data.append(go.Scatter(x=[None], y=[None], mode='markers',
                       marker=dict(size=10, color='red'),
                       legendgroup='Sell', showlegend=True, name='Sell'))
    
updatemenus = list([
    dict(active=-1,
         buttons=buttonlist,
    )
])
layout = dict(title='Feed', showlegend=True,
              updatemenus=updatemenus)

fig = dict(data=data, layout=layout)
iplot(fig)